# Clasificación multiclase

## Introduction

En este ejercicio se implementa la regresion one-vs-all y una red neuronal para reconocimiento de digitos.

Antes de empezar la ejecución de las partes de codigo correspondienters a los ejercicios, se requiere importar todas las librerias necesarias.

In [2102]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [2103]:
import pandas as pd
import random
#funcion para adaptar el dataset a mas de 10 000 datos.
def csvAdapted(path, modifyColumns, dropColumns):
    data = pd.read_csv(path, encoding='latin-1')

    
    columns = data.columns

    for index in modifyColumns:
        name = columns[index]
        value = list( data[name].unique() )
        data[name] = data[name].map( dict( zip( value,  [i for i in range( len(value) ) ] ) ) )

    data = data.drop([columns[index] for index in dropColumns ], axis=1)
    
    data=data.fillna(0)

    print(data.head())

    return data.to_numpy()

_data = csvAdapted("winequalityN.csv", [], [0] )


for index in range(len(_data)):
    if( _data[index][11] >=0 and  _data[index][11] <=3):
        _data[index][11]=0

    if( _data[index][11] >=4 and  _data[index][11] <=7):
        _data[index][11]=1

    if( _data[index][11] >=8):
        _data[index][11]=2


dataGenerate = []

for j in range(0,1):
    for i in range(0, 4000 ):
        aleatorio = random.randint(0,1)
        _data_ = [0,0,0,0,0,0,0,0,0,0,0,0]

        _data_[0] = _data[i][0] + random.randint(-1,1)
        _data_[1] = _data[i][1] + random.randint(-1,1)
        _data_[2] = _data[i][2] + random.randint(-1,1)
        _data_[3] = _data[i][3] + random.randint(-1,1)
   
        _data_[5] = _data[i][5] + random.randint(-1,1)
        _data_[6] = _data[i][6] + random.randint(-1,1)
        _data_[7] = _data[i][7] + random.randint(-1,1)
        _data_[8] = _data[i][8] + random.randint(-1,1)
        _data_[9] = _data[i][9] + random.randint(-1,1)
        _data_[10] = _data[i][10] + random.randint(-1,1)
        _data_[11] = _data[i][11] + random.randint(-1,1)

        if aleatorio>0.5:
            _data_[11] = _data[i][11] + 1
        else:
            _data_[11] = _data[i][11]
        dataGenerate.append( _data_ )

data =  np.append(_data,dataGenerate,axis= 0)
len(data)


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol  quality  
0      8.8        6  
1      9.5        6  
2     10.1        6 

10497

In [2104]:
#usaremos de nuestro dataset la columna 12 que seria la calidad para clasificarla en 3 categorias, donde: 1 es 'baja calidad', 2 es 'media calidad', 3 es 'buena calidad'
#3 etiquetas, de 1 a 3
num_labels = 3
#  datos de entrenamiento almacenados en los arreglos X, y , usando el 80% para el entrenamiento.
X, y = data[:8000, [0,1,2,3,4,5,6,7,8,9,10] ], data[:8000,11]
m = y.size

In [2105]:
print(X)
print(y)

[[ 7.    0.27  0.36 ...  3.    0.45  8.8 ]
 [ 6.3   0.3   0.34 ...  3.3   0.49  9.5 ]
 [ 8.1   0.28  0.4  ...  3.26  0.44 10.1 ]
 ...
 [ 8.4  -0.81 -0.51 ...  1.99 -0.68 12.  ]
 [ 8.3   1.2   1.49 ...  3.05  1.37  9.1 ]
 [ 5.6  -0.7   0.24 ...  2.13  1.36  8.2 ]]
[1. 1. 1. ... 1. 2. 2.]


In [2106]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [2107]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y 
    el gradiente del costo w.r.t. a los parámetros.
    
    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas 
        incluida la intercepcion
        
    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de 
        caracteristicas (incluida la intercepcion).
    
    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos
    
    lambda_ : float
        Parametro de regularización. 
    
    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada. 
    
    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la 
        función de costo con respecto a theta, en los valores actuales de theta..
    """
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [2108]:
J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 3.085728
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.355376, -0.491709, 0.885979, 1.663668]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


<a id="section2"></a>
### 1.4 Clasificacion One-vs-all
En esta parte del ejercicio, se implementará la clasificación de uno contra todos mediante el entrenamiento de múltiples clasificadores de regresión logística regularizados, uno para cada una de las clases $K$ en nuestro conjunto de datos. En el conjunto de datos de dígitos escritos a mano, $K = 10$, pero su código debería funcionar para cualquier valor de $K$.

El argumento `y` de esta función es un vector de etiquetas de 0 a 9. Al entrenar el clasificador para la clase $k \in \{0, ..., K-1 \} $, querrá un vector K-dimensional de etiquetas $y$, donde $y_j \ in 0, 1$ indica si la instancia de entrenamiento $j ^ {th}$ pertenece a la clase $k$ $(y_j = 1)$, o si pertenece a una clase diferente $(y_j = 0)$.

Además, se utiliza `optimize.minimize` de scipy para este ejercicio.
<a id="oneVsAll"></a>

In [2109]:
def oneVsAll(X, y, num_labels, lambda_):
    """
    
Entrena el numero de labels clasificadores de regresión logística y devuelve
    cada uno de estos clasificadores en una matriz all_theta, donde el i-ésimo
    fila de all_theta corresponde al clasificador para la etiqueta i.
    
    Parametros
    ----------
    X : tipo_array
        El conjunto de datos de entrada de forma (m x n). m es el número de
        puntos de datos, y n es el número de entidades. Tenga en cuenta que nosotros
        no asumimos que el término de intersección (o sesgo) está en X, sin embargo
        proporcionamos el código a continuación para agregar el término de sesgo a X.
    
    y : tipo_array
        Las etiquetas de datos. Un vector de forma (m, ).
    
    num_labels : entero
        numero posible de etiquetas
    
    lambda_ : real
        parametro regulador de la logistica
    
    Retorna
    -------
    all_theta : tipo_array
        los parámetros entrenados para la regresión logística para cada clase.
        Esta es una matriz de forma (K x n+1) donde K es el número de clases
        (es decir, `numlabels`) y n es el número de características sin el sesgo.
    """
    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 200}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == c), lambda_), 
                                jac=True, 
                                method='CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [2110]:
#Aqui podemos probar diferentes parametros para el regulador de la logistica(lambda)
lambda_ = 0.71



all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta)

/var/folders/h0/sdnxmh9s79j53gnv0s63splr0000gn/T/ipykernel_12431/1892987983.py:46: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
/var/folders/h0/sdnxmh9s79j53gnv0s63splr0000gn/T/ipykernel_12431/1892987983.py:46: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
/var/folders/h0/sdnxmh9s79j53gnv0s63splr0000gn/T/ipykernel_12431/1892987983.py:46: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


[[-0.50334303  0.15536474  0.98408475 -0.14658911 -0.06621669  0.15933694
   0.03174221 -0.00996122 -0.14342038 -0.36327402 -0.65944584 -0.42922609]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [-0.38695292 -0.18272553 -0.20134753  0.10762299 -0.01719408 -0.86497517
  -0.00511143  0.00594607 -0.09760296 -0.48993693  0.02014062  0.08370154]]


<a id="section3"></a>
#### 1.4.1 Prediccion One-vs-all

Después de entrenar el clasificador de one-vs-all, se puede usarlo para predecir el dígito contenido en una imagen determinada. Para cada entrada, debe calcular la "probabilidad" de que pertenezca a cada clase utilizando los clasificadores de regresión logística entrenados. La función de predicción one-vs-all seleccionará la clase para la cual el clasificador de regresión logística correspondiente genera la probabilidad más alta y devolverá la etiqueta de clase (0, 1, ..., K-1) como la predicción para el ejemplo de entrada.

In [2111]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas. 
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase. 
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1] 
    predice clases 0, 2, 0, 1 para 4 ejemplos).
    
    Parametros
    ----------
    all_theta : tipo_array
     
        Los parámetros entrenados para la regresión logística para cada clase.
        Esta es una matriz de forma (K x n+1) donde K es el número de clases
        y n es el número de características sin el sesgo.
    X : tipo_array
        Puntos de datos para predecir sus etiquetas. Esta es una matriz de forma
        (m x n) donde m es el número de puntos de datos para predecir, y n es el número
        de características sin el término de sesgo. Tenga en cuenta que agregamos el término de sesgo para X en
        esta función
    
    Devuelve
    -------
    p : array_like
        Las predicciones para cada punto de datos en X. Este es un vector de forma (m, ).
    """
    
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Aggrega ones a la matriz de datos X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

Una vez que haya terminado, se llama a la función `predictOneVsAll` usando el valor aprendido de $\theta$. Debería apreciarse que la precisión del conjunto de entrenamiento es de aproximadamente 95,1% (es decir, clasifica correctamente el 95,1% de los ejemplos del conjunto de entrenamiento).

In [2112]:
#ingresamos el 20% de datos restantes de nuestro data set para usarlos como comparacion
XPrueba = data[8000:,[0,1,2,3,4,5,6,7,8,9,10] ].copy()

pred = predictOneVsAll(all_theta, XPrueba)
# imprimimos la prediccion y la precision del conjunto de entrenamiento
print(pred)

print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == data[8000:,11]) * 100))



[1 1 1 ... 1 1 1]
Precision del conjuto de entrenamiento: 48.14%
